# Linear Sorting Algorithms - Complete Guide

### 🎯 The Big Question
**Can we sort faster than O(n log n)?**

Last time we achieved faster `find()` operations using direct access arrays and hash tables. Now let's see if we can achieve faster `sort()` too!

---

## 1. Review: Data Structure Performance Overview

From the lecture, here's where we stand with different data structures:

| Data Structure | build(X) | find(k) | insert(x) | delete(k) | find_min() | find_max() | find_prev(k) | find_next(k) |
|----------------|----------|---------|-----------|-----------|------------|------------|--------------|--------------|
| **Array** | n | n | n | n | n | n | n | n |
| **Sorted Array** | n log n | log n | n | n | 1 | 1 | log n | log n |
| **Direct Access Array** | u | 1 | 1 | 1 | u | u | u | u |
| **Hash Table** | n(e) | 1(e) | 1(a)(e) | 1(a)(e) | n | n | n | n |

*(e) = expected time, (a) = amortized time*

**Key Insight**: We got faster find using random access indexing - an operation with **linear branching factor** instead of the binary branching in comparison trees!

---

## 2. Why Comparison Sorts Are Limited

### The Comparison Sort Lower Bound Proof

**Step 1**: Comparison model means algorithm decision tree is **binary** (constant branching factor)

**Step 2**: Number of leaves L ≥ number of possible outputs

**Step 3**: Tree height is lower bounded by Ω(log L), so worst-case running time is Ω(log L)

**Step 4**: To sort array of n elements, number of outputs = **n! permutations**

**Step 5**: Height ≥ log(n!) ≥ log((n/2)^(n/2)) = Ω(n log n)

**Conclusion**: Merge sort is **optimal in the comparison model**

### 💡 The Breakthrough Idea
Can we exploit a **direct access array** to sort faster by avoiding comparisons entirely?

## 3. Direct Access Array Sort

### The Basic Algorithm
**Assumption**: All keys are unique non-negative integers in range {0, ..., u-1}, so n ≤ u

**Strategy**:
1. Insert each item into a direct access array of size u → **Θ(n)**
2. Return items in order they appear in direct access array → **Θ(u)**
3. **Total running time**: Θ(u), which is **Θ(n) if u = Θ(n)**

### Example: [5, 2, 7, 0, 4]

```
Input:  [5, 2, 7, 0, 4]
         ↓ ↓ ↓ ↓ ↓
Index:   0 1 2 3 4 5 6 7
Array: [0] _ [2] _ [4] [5] _ [7]
         ↓     ↓     ↓  ↓     ↓
Output: [0, 2, 4, 5, 7]  ← Read left to right
```

```python
def direct_access_sort(A):
    """Sort A assuming items have distinct non-negative keys"""
    
    # Step 1: Find maximum key to determine array size
    u = 1 + max([x.key for x in A])  # O(n) - scan through all items
    
    # Step 2: Create direct access array of size u
    D = [None] * u  # O(u) - all positions initially empty
    
    # Step 3: Insert each item at position = its key value
    for x in A:  # O(n) - go through each item once
        D[x.key] = x  # Direct placement using key as index
    
    # Step 4: Read back items in sorted order
    i = 0
    for key in range(u):  # O(u) - check every possible position
        if D[key] is not None:  # If position has an item
            A[i] = D[key]  # Add it to output
            i += 1
```

**Time Complexity**: O(n + u)  
**Space Complexity**: O(u)  
**Perfect when**: u = Θ(n)

### ❌ Problem: What if u is much larger?
What if keys are in larger range, like **u = Ω(n²)** or even **u < n²**?

Example: Sorting [17, 3, 24, 22, 12] would need an array of size 25, but we only have 5 elements!

---

In [ ]:
def direct_access_sort(A: List[Item]) -> List[Item]:
    """Sort A assuming items have distinct non-negative keys"""
    print(f"🔄 DIRECT ACCESS ARRAY SORT")
    print(f"Input: {A}")
    
    # Step 1: Find maximum key to determine array size
    u = 1 + max([x.key for x in A])  # O(n) - scan through all items
    print(f"Maximum key range needed: {u}")
    
    # Step 2: Create direct access array of size u
    D = [None] * u  # O(u) - all positions initially empty
    print(f"Created direct access array of size {u}")
    
    # Step 3: Insert each item at position = its key value
    for x in A:  # O(n) - go through each item once
        D[x.key] = x  # Direct placement using key as index
        print(f"  Placed {x} at index {x.key}")
    
    # Step 4: Read back items in sorted order
    i = 0
    result = []
    for key in range(u):  # O(u) - check every possible position
        if D[key] is not None:  # If position has an item
            A[i] = D[key]  # Add it to output
            result.append(D[key])
            i += 1
    
    print(f"✅ Output: {result}")
    print(f"Time: O(n + u) = O({len(A)} + {u})")
    return A

# Example from lecture: [5, 2, 7, 0, 4]
items = [Item(5), Item(2), Item(7), Item(0), Item(4)]
sorted_items = direct_access_sort(items.copy())

## 4. Handling Large Key Ranges: The Tuple Conversion Trick

### 💡 Key Idea
**Represent each key k by tuple (a, b) where k = a×n + b and 0 ≤ b < n**

- Specifically: a = ⌊k/n⌋ < n and b = (k mod n)
- This is just a **2-digit base-n number**!
- Python built-in: `(a, b) = divmod(k, n)`

### Real Example from Your Whiteboard: [17, 3, 24, 22, 12] with n=5

```python
# Step 1: Convert each key to (a,b) tuple where k = a*n + b
n = 5

# For key 17: divmod(17, 5) = (3, 2) because 17 = 3*5 + 2
# For key 3:  divmod(3, 5)  = (0, 3) because 3 = 0*5 + 3  
# For key 24: divmod(24, 5) = (4, 4) because 24 = 4*5 + 4
# For key 22: divmod(22, 5) = (4, 2) because 22 = 4*5 + 2
# For key 12: divmod(12, 5) = (2, 2) because 12 = 2*5 + 2

original_keys = [17, 3, 24, 22, 12]
tuples = [(3,2), (0,3), (4,4), (4,2), (2,2)]
```

**Visual representation**:
```
[17, 3, 24, 22, 12]  →  [(3,2), (0,3), (4,4), (4,2), (2,2)]  →  [32, 03, 44, 42, 22]
```

### 🔑 Critical Question: How to sort tuples?
We want **lexicographic order**: first compare `a` values, then compare `b` values for ties.

**Key Insight**: Sort them in **least significant to most significant** order!

---

## 5. Tuple Sort: The Step-by-Step Process

### Exercise from PDF: [32, 03, 44, 42, 22] with n=5

Let's trace through this **exactly as shown in the PDF**:

**Step 1**: Sort by **second digit (b)** - the **least significant**
```
Original: [32, 03, 44, 42, 22]
         b: 2   3   4   2   2

Sort by b: [32, 42, 22, 03, 44]
           b: 2   2   2   3   4
```

**Step 2**: Sort by **first digit (a)** - the **most significant**
```
From Step 1: [32, 42, 22, 03, 44]
           a:  3   4   2   0   4

Sort by a: [03, 22, 32, 42, 44]
           a:  0   2   3   4   4
```

**Final result**: [03, 22, 32, 42, 44] which corresponds to [3, 12, 17, 22, 24] ✅

### Why Least-Significant-First Works

The **stability** property is crucial here:
- When we sort by `a` in Step 2, items with the same `a` value stay in the same relative order from Step 1
- Since Step 1 already sorted by `b`, items with same `a` are automatically in correct `b` order!

### 🚨 Problem: Direct Access Array Sort Can't Handle Duplicates!

Look at our tuple example - we have multiple items with `b = 2`: (3,2), (4,2), (2,2)

**Direct access array sort fails** because it can only store ONE item per index!

We need a sorting algorithm that:
- **Allows repeated keys** 
- **Preserves input order** (stability)

---


## 6. Counting Sort: The Solution to Duplicates

### Key Innovation
Instead of storing a **single item** at each array index, store a **chain (list)** - just like hashing!

### Requirements for Stability
- Chain data structure must **remember the order** items were added
- Use a **sequence data structure** which maintains insertion order
- To insert item x: insert **last** to end of chain at index x.key
- To sort: read through all chains in sequence order

```python
def counting_sort(A):
    """Sort A assuming items have non-negative keys"""
    
    # Step 1: Find maximum key to determine array size
    u = 1 + max([x.key for x in A])  # O(n) - scan all items
    
    # Step 2: Create direct access array of empty chains
    D = [[] for i in range(u)]  # O(u) - list of empty lists
    
    # Step 3: Insert each item into chain at its key position
    for x in A:  # O(n) - process each item once
        D[x.key].append(x)  # Add to END of chain (preserves order!)
    
    # Step 4: Read out items in order
    i = 0
    for chain in D:  # O(u) - check each possible key value
        for x in chain:  # O(length of chain) - read each item
            A[i] = x
            i += 1
    # Total time for Step 4: O(u + n) since sum of chain lengths = n
```

### Why This Works
- **Time**: O(n + u) total
- **Space**: O(n + u) for the chains
- **Stable**: ✅ Items with same key maintain original order
- **Handles duplicates**: ✅ Multiple items can have same key

### Counting Sort Example
```
Input with duplicates: [2, 1, 3, 1, 2]

D[0]: []
D[1]: [1st item with key=1, 2nd item with key=1]  ← preserves order
D[2]: [1st item with key=2, 2nd item with key=2]  ← preserves order  
D[3]: [item with key=3]

Output: [1, 1, 2, 2, 3] with original relative order preserved
```

---

## 7. Radix Sort: Putting It All Together

### The Complete Algorithm

**Goal**: If u < n², use **tuple sort** with **auxiliary counting sort** to sort tuples (a, b)

**Strategy**:
1. Convert large keys to base-n tuples
2. Sort **least significant key b**, then **most significant key a**  
3. **Stability ensures** previous sorts stay sorted
4. **Running time**: O(2n) = O(n) ✅

### Generalization: Beyond 2 Digits

**Key insight from PDF**: If every key < n^c for some positive constant c = log_n(u), then:
- Every key has **at most c digits** in base n
- A c-digit number can be written as **c-element tuple** in O(c) time
- We sort each of the **c base-n digits** in O(n) time using counting sort
- **Total time**: O(c×n) time
- **If c is constant**, so each key ≤ n^c, this sort is **linear O(n)**!

### Complete Radix Sort Implementation

```python
def radix_sort(A):
    """Sort A assuming items have non-negative keys"""
    
    n = len(A)
    
    # Step 1: Find maximum key
    u = 1 + max([x.key for x in A])  # O(n)
    
    # Step 2: Calculate number of digits needed in base n
    # This is c = log_n(u), computed as bit_length ratios
    c = 1 + (u.bit_length() // n.bit_length())
    
    # Step 3: Create wrapper objects to hold digit representations
    class Obj: pass  # Simple class to hold attributes
    D = [Obj() for a in A]  # One wrapper per input item
    
    # Step 4: Convert each key to c-digit tuple in base n
    for i in range(n):  # O(n×c) total
        D[i].digits = []  # Will store [least_sig, ..., most_sig]
        D[i].item = A[i]  # Remember original item
        
        high = A[i].key  # Start with full key value
        for j in range(c):  # Extract c digits - O(c)
            high, low = divmod(high, n)  # Split off rightmost digit
            D[i].digits.append(low)  # Add to digit list
        # After loop: digits[0] = least significant, digits[c-1] = most significant
    
    # Step 5: Sort by each digit, from least to most significant
    for i in range(c):  # c passes of counting sort - O(c×n)
        
        # Set the key for this pass to digit i
        for j in range(n):  # O(n)
            D[j].key = D[j].digits[i]  # Use digit i as sorting key
        
        # Sort by this digit using counting sort
        counting_sort(D)  # O(n) - stable sort on current digit
    
    # Step 6: Extract final sorted items
    for i in range(n):  # O(n)
        A[i] = D[i].item
```

### Code Explanation - Key Parts

**Line 6**: `c = 1 + (u.bit_length() // n.bit_length())`
- `u.bit_length()` = number of bits needed to represent u
- `n.bit_length()` = number of bits needed to represent n  
- This computes ⌈log_n(u)⌉ = number of base-n digits needed

**Lines 12-17**: Converting to digit representation
- `divmod(high, n)` splits off the rightmost base-n digit
- We build digits from least to most significant (index 0 = least significant)

**Lines 20-24**: The core sorting loop
- Sort by digit 0 (least sig), then digit 1, ..., then digit c-1 (most sig)
- Each call to `counting_sort` is stable, preserving previous ordering

### Time Complexity Analysis

**Total running time**: O(n + n×log_n(u))
- When u ≤ n^c for constant c: log_n(u) ≤ c, so time is **O(n)**
- **This breaks the Ω(n log n) comparison sort lower bound!**

---

## 8. Complete Algorithm Comparison Table (from PDF)

| Algorithm | Time O(·) | In-place? | Stable? | Comments |
|-----------|-----------|-----------|---------|----------|
| **Insertion Sort** | n² | Y | Y | O(nk) for k-proximate |
| **Selection Sort** | n² | Y | N | O(n) swaps |
| **Merge Sort** | n log n | N | Y | stable, optimal comparison |
| **Counting Sort** | n + u | N | Y | O(n) when u = O(n) |
| **Radix Sort** | n + n log_n(u) | N | Y | O(n) when u = O(n^c) |

### Key Properties Explained

**In-place**: 
- Y = Uses O(1) extra space
- N = Uses O(n) or more extra space

**Stable**: 
- Y = Items with equal keys maintain their relative order
- N = Relative order of equal items may change

**Comments**:
- **k-proximate**: Array where each element is at most k positions away from its sorted position
- **Optimal comparison**: Achieves the theoretical lower bound for comparison-based sorting
- **u = O(n)**: Key range is proportional to number of elements
- **u = O(n^c)**: Key range is polynomial in number of elements

---

## 9. When to Use Each Algorithm

### Direct Access Array Sort
✅ **Use when**: 
- Keys are unique non-negative integers
- Key range u = O(n)
- Need O(n) sorting

❌ **Don't use when**:
- Have duplicate keys
- Key range is large (u >> n)

### Counting Sort  
✅ **Use when**:
- Keys are non-negative integers (can have duplicates)
- Key range u = O(n) 
- Need stable sorting
- Building histograms or frequency counts

❌ **Don't use when**:
- Key range is very large
- Memory is severely constrained

### Radix Sort
✅ **Use when**:
- Keys are integers with u ≤ n^c for small constant c
- Need linear time sorting
- Stability is important
- Sorting strings (treat as base-256 numbers)

❌ **Don't use when**:
- Keys are arbitrary objects with complex comparison
- Key range grows exponentially with n
- Working with floating-point numbers requiring exact comparison

---

## 10. Key Insights and Takeaways

### 🎯 The Fundamental Breakthrough
**Linear sorting algorithms break the O(n log n) barrier** by:
1. **Avoiding comparisons entirely** - use key values directly as array indices
2. **Exploiting structure in the data** - integer keys in bounded ranges
3. **Using stability creatively** - sort multiple digits in correct order

### 🔑 Essential Concepts
1. **Comparison model limitation**: Binary decision trees → Ω(n log n) lower bound
2. **Direct access power**: Linear branching factor breaks the barrier  
3. **Base conversion trick**: Large ranges → multiple small ranges
4. **Stability preservation**: Least-significant-first digit sorting
5. **Space-time tradeoff**: O(n) extra space for O(n) time

### 💡 Practical Applications
- **Database sorting** with integer keys
- **String sorting** (radix sort on ASCII values)  
- **Counting/histogram** operations
- **Preprocessing** for other algorithms
- **External sorting** of large datasets